# cython, numba 라이브러리를 사용해 성능 향상시키기

## 코드의 성능을 향상시켜 실행 시간 측정하기 ― timeit

timeit은 표현식의 실행 시간을 측정할 때 사용하는 '파이썬 문장'이다.

timeit은 실행 시간을 측정해야 하는 코드 앞에 %%time 이라고 작성하면 된다.

이는 주로 성능을 저하시키는 부분이 어느 곳인지 확인하기 위해 자주 사용한다.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'a': [10, 20, 30], 'b': [20, 30, 40]})

In [7]:
def avg_2_apply(row):
    print(row)
    x = row[0]
    y = row[1]
    print(x,y)
    if(x == 20):
        return np.nan
    else:
        return (x + y)/2

In [8]:
df.apply(avg_2_apply, axis = 1)

a    10
b    20
Name: 0, dtype: int64
10 20
a    20
b    30
Name: 1, dtype: int64
20 30
a    30
b    40
Name: 2, dtype: int64
30 40


0    15.0
1     NaN
2    35.0
dtype: float64

### 2. 판다스 데이터프레임 ― 실행 시간 측정

In [10]:
def avg_2_apply(row):
    x = row[0]
    y = row[1]
    if(x == 20):
        return np.nan
    else:
        return (x + y)/2

avg_2_apply 함수를 총 1000번 반복해서 실행한 결과 실행 시간은 735마이크로초이다

1 마이크로초는 1초를 백만으로 나눈 값이다.

In [11]:
%%timeit
df.apply(avg_2_apply, axis = 1)

735 µs ± 24.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### 3. 넘파이로 벡터화한 함수 사용하기 ― 실행 시간 측정

넘파이로 함수를 백터화한 다음 실행 시간을 측정해보자.

In [12]:
@np.vectorize
def v_avg_2mod(x, y):
    if(x == 20):
        return (np.NaN)
    else:
        return (x + y) / 2

v_avg_2_mod 함수는 총 10,000회 반복 시행되었으며 실행 시간은 66.3 마이크로초이다.

1 나노초(ns)는 1 마이크로초(µs)를 1,000으로 나눈 값이다.

In [14]:
%%timeit
v_avg_2mod(df['a'], df['b'])

66.3 µs ± 2.41 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### 5. numba 라이브러리로 벡터화한 함수 사용하기 ― 실행 시간 측정

아래는 numba 라이브러리로 함수를 벡터화한 것이다.

In [15]:
import numba

@numba.vectorize
def v_avg_2_numba(x, y):
    if(x == 20):
        return (np.NaN)
    else:
        return (x + y) / 2

v_avg_2_numba 함수는 총 10,000회 반복했고 실행 시간은 12 마이크로초이다.

함수를 최적화하면 프로그램의 성능이 향상된다.

numba 라이브러리로 벡터화한 함수가 가장 빠른 연산 속도를 보여준다.

In [16]:
%%timeit
v_avg_2_numba(df['a'].values, df['b'].values)

The slowest run took 8.71 times longer than the fastest. This could mean that an intermediate result is being cached.
12 µs ± 15 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
